In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
#!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
#!cp mystem /bin
from pymystem3 import Mystem
m = Mystem()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

%matplotlib inline

Считываем данные из таблиц

In [5]:
all_queries = pd.read_csv('all_queries.csv', sep='\t')
all_queries.sample(10)

,searchterm,count
194229,629822b5,1
186962,привязь страховочная пятиточечная с фалом,1
9479,счётчик электроэнергии меркурий,6
46573,конвектор patriot,2
163715,бензиновый двигатель лифан,1
105297,биты stayer,1
214351,струубцина,1
137419,20 550,1
90445,краска аэрозольная coralino с30318 металлик яр...,1
128229,комплект постельного белья narcissa gr164956 с...,1


In [6]:
redirects = pd.read_csv('redirects.tsv', sep='\t')
redirects.sample(10)

,query,url
14521,multitronics компьютер маршрутн,catalog/bortovoi-marshrutnii-computer/?f=br__2...
30374,перфоратор сверл,catalog/bury/
29174,маск сварочн тсс,catalog/svarochnye-maski/?f=br__1168#h1
29623,мотокос ремн триммер,catalog/remni-dlya-trimmerov/
17784,septicsol биогранул,catalog/zhidkost-dlja-biotualeta/?f=br__354#h1
7420,frybest для запекан форм,catalog/vse-dlya-vypechki/?f=br__1182#h1
18795,stealth,producer/stealth/
5189,echo колпак,catalog/8-771/0/?f=br__28#h1
6903,fit инструмент проч ручн,catalog/prochij-ruchnoj-instrument/?f=br__444#h1
3816,chefout,/producer-1934/


В задании сказано, что в ответе необходимо получить соответстивя для запросов, которые встретились не меньше трех раз, так что для упрощения работы отфильтруем запросы сразу (не обнаружила в задании условий, по которым нам они будут необходимы в задаче).

In [7]:
queries_filtered = all_queries[all_queries['count']>=3].copy().reset_index(drop=True)

Проверим на наличие пустых элементов.

In [8]:
print(queries_filtered[queries_filtered['searchterm'].isna()], end='\n\n')
print(redirects[redirects['query'].isna()])

  searchterm   count
0        NaN  134696

Empty DataFrame
Columns: [query, url]
Index: []


In [9]:
queries_filtered.drop(index=[0], inplace=True)

После изучения данных в таблицах было обнаружено, что текстовые данные в таблице с редиректами уже предобработаны, притом не лемматизированы, а являются результатами стемминга. Для того чтобы учесть все варианты, применим оба алгоритма к данным таблицы запросов из базы.

In [10]:
# создаем колонку для лемматизированных текстов, используем лемматизатор pymystem, сохраняем все токены из букв и цифр
queries_filtered['lem'] = queries_filtered['searchterm'].apply(lambda x: ' '.join([l for l in m.lemmatize(x.lower()) if not l.isspace()]) if type(x) is str else str(x))

# создаем колонку для результатов стемминга, используем nltk SnowballStemmer для русского языка, сохраняем все токены из букв и цифр
queries_filtered['stem'] = queries_filtered['searchterm'].apply(lambda x: ' '.join(
    [stemmer.stem(t) for t in x.lower().split()]) if type (x) is str else str(x))

# Эксперимент первый: лемматизация

In [10]:
#обучаем бинарный CountVectorizer для получения модели Bag-of-words, учитывающей только вхождение или невхождение токена в текст
vectorizer = CountVectorizer(binary=True)
vectorizer.fit(queries_filtered['lem'].append(redirects['query']))

# получаем вектора текстов для редиректов и лемматизированных запросов
redirects_vec = vectorizer.transform(redirects['query'])
queries_vec = vectorizer.transform(queries_filtered['lem'])

In [11]:
# строим матрицу схожести используя косинусную близость
match_matrix = cosine_similarity(queries_vec, redirects_vec, dense_output=True)
# если косинусная близость равна единице, векторы полностью совпали, запросы одинаковы, запоминаем маску
matches = np.where(match_matrix==1)
# получаем датафрейм с результатами, используя индексы запросов и редиректов
res_lem = pd.DataFrame({'query': list(queries_filtered['searchterm'].iloc[matches[0]]),
                        'url': list(redirects['url'].iloc[matches[1]])}).drop_duplicates(ignore_index=True)

res_lem.to_csv('/content/gdrive/MyDrive/diginetica/res_lem.csv')
res_lem

,query,url
0,респиратор,catalog/respiratory/
1,шуруповерт,catalog/akkumulyatornye-dreli-shurupoverty/
2,компрессор,catalog/kompressory/
3,генератор,catalog/5-81/
4,степлер,catalog/steplery/
...,...,...
546,nws,producer/nws/
547,крона,/producer-3073/
548,тросорез,catalog/trosorezy/
549,тример,search/?q=%f2%f0%e8%ec%ec%e5%f0


# Эксперимент второй: стемминг
Шаги те же, что в первом

In [11]:
vectorizer = CountVectorizer(binary=True)
vectorizer.fit(queries_filtered['stem'].append(redirects['query']))

redirects_vec = vectorizer.transform(redirects['query'])
queries_vec = vectorizer.transform(queries_filtered['stem'])

In [12]:
match_matrix = cosine_similarity(queries_vec, redirects_vec, dense_output=True)
matches = np.where(match_matrix==1)
res_stem = pd.DataFrame({'query': list(queries_filtered['searchterm'].iloc[matches[0]]),
                        'url': list(redirects['url'].iloc[matches[1]])}).drop_duplicates(ignore_index=True)

res_stem.to_csv('/content/gdrive/MyDrive/diginetica/res_stem.csv')
res_stem

,query,url
0,респиратор,catalog/respiratory/
1,шуруповерт,catalog/akkumulyatornye-dreli-shurupoverty/
2,болгарка,catalog/2-46/0/
3,компрессор,catalog/kompressory/
4,генератор,catalog/5-81/
...,...,...
713,тросорез,catalog/trosorezy/
714,трещеткой,catalog/vorotki-privodi-treschetki/
715,тример,search/?q=%f2%f0%e8%ec%ec%e5%f0
716,мастер,producer/master/


# Финальная таблица

In [2]:
res_stem = pd.read_csv('res_stem.csv', index_col=0)
res_lem = pd.read_csv('res_lem.csv', index_col=0)

In [3]:
print('Number of results after stemming: {}'.format(res_stem.shape[0]))
print('Number of results after lemmatization: {}'.format(res_lem.shape[0]))

full = res_stem.append(res_lem, ignore_index=True)

print('Number of intersecting elements: {}'.format(full.duplicated().sum()))
print('{} unique stem and {} unique lem elements left'.format(res_stem.shape[0]-full.duplicated().sum(), res_lem.shape[0]-full.duplicated().sum()))

full.drop_duplicates(inplace=True, ignore_index=True)
full.to_csv('/content/gdrive/MyDrive/diginetica/result.csv')

full

Number of results after stemming: 718
Number of results after lemmatization: 551
Number of intersecting elements: 436
282 unique stem and 115 unique lem elements left


,query,url
0,респиратор,catalog/respiratory/
1,шуруповерт,catalog/akkumulyatornye-dreli-shurupoverty/
2,болгарка,catalog/2-46/0/
3,компрессор,catalog/kompressory/
4,генератор,catalog/5-81/
...,...,...
828,сантехнить,/producer-1659/
829,olfa,producer/olfa/
830,тольятти,/producer-3375/
831,itelma,/producer-1627/
